In [16]:
import numpy as np
#from multivar_horner import HornerMultivarPolynomial
import sympy as sp

# Population polynomial - polynomial.f90 : newton

The subroutine calculates the solution for n d-dimensional polynomial equations using the Newton-Raphson method in combination with the Horner method for multivariate polynomials. 
The Horner method is used for the calculation of the value of the polynomial and its derivatives at a given point.

#### $\textbf{Newton-Raphson method using derivatives}$
The Newton-Raphson method is an iterative method for finding the roots of a function. The method requires both the function $f(x)$ and the derivative $f'(x)$ at arbitrary points $x$. The formula consists geometrically of extending the tangent line at a current point $x_i$ until it intersects the x-axis. The intersection point is the next guess $x_{i+1}$. The formula for the one dimensional case is given by: </p>
$$x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)}$$
This can be expressed in terms of a multiplication of the inverse of the Jacobian matrix with the function vector. The Jacobian matrix is the matrix of all first-order partial derivatives of the function. The formula for the multivariate case is given by: </p>
$$x_{i+1} = x_i - J_f^{-1}(x_i) f(x_i)$$
where $J$ is the Jacobian matrix and $f(x_i)$ is the function vector. The Jacobian matrix is given by: </p>
$$J_f = \begin{bmatrix} \frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & \cdots & \frac{\partial f_1}{\partial x_n} \\ \frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} & \cdots & \frac{\partial f_2}{\partial x_n} \\ \vdots & \vdots & \ddots & \vdots \\ \frac{\partial f_n}{\partial x_1} & \frac{\partial f_n}{\partial x_2} & \cdots & \frac{\partial f_n}{\partial x_n} \end{bmatrix}$$

The subroutine calculates the inverse of the Jacobian matrix directly for the dimensions one, two and three. For higher dimensions, the subroutine uses the LU decomposition to calculate the inverse of the Jacobian matrix. The LU decomposition is a method for solving systems of linear equations. The method decomposes the matrix into a lower triangular matrix and an upper triangular matrix. The inverse of the Jacobian matrix is then calculated by solving the linear system for each column of the identity matrix. (Replace by LAPACK?) </p> 

#### $\textbf{Horner method for multivariate polynomials}$
The value of the function and its derivatives at a given point are calculated using the Horner method. The Horner method is a method for evaluating polynomials. The method is based on the factorization of the polynomial. In the one-dimensional case, the polynomial is written as: </p>
$$p(x) = a_0 + a_1 x + a_2 x^2 + \cdots + a_n x^n = a_0 + x(a_1 + x(a_2 + \cdots + x(a_{n-1} + a_n x)))$$
The derivative of the polynomial is calculated in a similar way using the chain rule. The multivariate polynomial is written as: </p>
$$p(x_1, x_2, \cdots, x_n) = a_0 + a_1 x_1 + a_2 x_1^2 + \cdots + a_n x_1^{n_1} + a_{n+1} x_2 + a_{n+2} x_1 x_2 + \cdots + a_{n+m} x_1^{n_1} x_2^{n_2} \cdots x_m^{n_m}$$
This is achieved by recursively evaluating the polynomial for each variable. </p></br>

#### $\textbf{Explicit usage}$
One of the main tasks of the peacemaker code is the determination of the set of equilibrium cluster populations $\{N_i\}$. In the canonical ensemble, the equilibrium populations are determined by the minimization of the free energy (Helmholtz energy). The free energy is given by: </p>
$$F = -k_{\text{B}}T \cdot \ln{Q}$$
where $Q$ is the partition function. The partition function is given by: </p>
$$ Q = \prod_{i=1}^L \frac{1}{N_i!} q_i^{N_i}$$
where $q_i$ is the partition function of the cluster $i$ and $L$ is the the total number of clusters in the clusterset. The partition function of the cluster $i$ is given by: </p>
$$ q_i = q_{\text{trans}}~q_{\text{elec}}~q_{\text{vib}}~q_{\text{rot}}$$
Thus, using Stirling's approximation: $\ln{N!} = N \ln{N} - N$, the Helmholtz energy can be expressed as: </p>
$$\begin{aligned} F  &= -k_{\text{B}}T \cdot \ln{\prod_{i=1}^L \frac{1}{N_i!} q_i^{N_i}} \\
     &= -k_{\text{B}}T \cdot \sum_{i=1}^L \ln{q_i^{N_i}} + \ln{\frac{1}{N_i!}} \\
     &= -k_{\text{B}}T \cdot \sum_{i=1}^L N_i\ln{q_i} - \ln{N_i!}\\
     &= -k_{\text{B}}T \cdot \sum_{i=1}^L N_i\ln{q_i} - N_i\ln{N_i}+N_i \end{aligned}$$
The expression has to be minimized with respect to the set of equilibrium populations $\{N_i\}$. </p>
$$ \begin{aligned}  &(1)~~~\frac{\partial F}{\partial N_i} = -k_{\text{B}}T \cdot \bigg( \ln{q_i} - \ln{N_i} \bigg) = -k_{\text{B}}T\ln{\frac{q_i}{N_i}} = 0 \\
                    &(2)~~~\frac{\partial F}{\partial N_i} = \sum_{c=1}^K n_i^c \frac{\partial F}{\partial N_c} = -k_{\text{B}}T \cdot \sum_{c=1}^{K} n_i^c \frac{\partial}{\partial N_c} \bigg( \sum_{i=1}^L N_i\ln{q_i} - N_i\ln{N_i}+N_i \bigg) = -k_{\text{B}}T \sum_{c=1}^K n_i^c \ln{\frac{q_c}{N_c}} = 0 \end{aligned}  $$


$$ \begin{aligned} (1) = (2) ~~~~&\Leftrightarrow~~~~ \sum_{c=1}^{K} n_i^c \ln{\frac{q_c}{N_c}} = \ln{\frac{q_i}{N_i}} \\
                                 &\Leftrightarrow~~~~ N_i = q_i \prod_{c=1}^{K} \bigg( \frac{N_c}{q_c} \bigg)^{n_i^c} \end{aligned} $$

With $K$ being the total number of components in the system. </br>
This reduces the problem to finding the the monomer populations $N_c$, since the cluster polulations can be determined from them. 
For doing this, the conservation of the total number of atoms has to be taken into account, to construct the population polynomial. The conservation of the total number of atoms is given by: </p>
$$ \begin{aligned} \sum_c^K N_c^{\text{tot}} &= \sum_{i=1}^L \sum_{c=1}^{K} n_i^c N_i \\
\Leftrightarrow~~~~~~ 0 &= \frac{\sum_{i=1}^L \sum_{c=1}^{K} n_i^c N_i}{\sum_c^K N_c^{\text{tot}}} - 1 \\
                        &= \frac{1}{\sum_c^K N_c^{\text{tot}}} \sum_{i=1}^L \sum_{c=1}^{K} n_i^c \prod_c^K \bigg[ q_i \bigg( \frac{N_c}{q_c} \bigg)^{n_i^c} \bigg] - 1 \end{aligned} $$
where $N_{\text{tot}}$ is the total number of atoms in the system. </p>

For soving an polynomial with $N_c$ unknowns, as many equations as unknowns are needed. The quantity Z is introduced. It does not have to represent any physical quantity. </p>
$$\textbf{n} = \begin{bmatrix} n_1^a & n_2^a & \cdots & n_L^a \\ n_1^b & n_2^b & \cdots & n_L^b \\ \vdots & \vdots & \ddots & \vdots \\ n_1^K & n_2^K & \cdots & n_L^K \end{bmatrix}; ~~~~~
   \vec{N} = \begin{bmatrix} N_1 \\ N_2 \\ \vdots \\ N_L \end{bmatrix}; ~~~~~
   \vec{N^{\text{tot}}} = \begin{bmatrix} N_a^{\text{tot}} \\ N_b^{\text{tot}} \\ \vdots \\ N_K^{\text{tot}} \end{bmatrix}; ~~~~~
   \vec{Z} = \begin{bmatrix} Z_1 \\ Z_2 \\ \vdots \\ Z_K \end{bmatrix}$$
   It is obvious that: </p>
$$ \textbf{n} \cdot \vec{N} = \vec{N^{\text{tot}}}$$
and thus: </p>
$$ \vec{Z}^T \cdot \textbf{n} \cdot \vec{N} = \vec{Z}^T \vec{N^{\text{tot}}} $$
Using the same rearrangement as above, the equation can be written as: </p>
$$ 0 = \frac{1}{\sum_c^K Z_c N_c^{\text{tot}}} \sum_{i=1}^L \sum_{c=1}^{K} Z_c n_i^c \prod_c^K \bigg[ q_i \bigg( \frac{N_c}{q_c} \bigg)^{n_i^c} \bigg] - 1 $$  


The unknowns are the monomer populations $N_c$ and the powers are the number of particles of the corresponding component in the cluster.
The rest is calculated as coefficients of the polynomial. In $\textbf{Peacemaker}$ the -1 is obtained by setting the coefficient of the part with degree zero for all components to -1. </p>
For a binary system with $K=2$ components, the population polynomial is given by: </p>
$$  0 = \sum_{i=1}^L \frac{ n_i^a + n_i^b}{N_{\text{tot}}} \cdot q_i \bigg( \frac{N_a}{q_a} \bigg)^{n_i^a} \bigg( \frac{N_b}{q_b} \bigg)^{n_i^b} - 1 $$
which can be written as ($N_a = x, N_b = y$): </p>
$$ 0 = \sum_{i=1}^L c_i \cdot \text{x}^{n_i^a} \text{y}^{n_i^b} -1 $$
For every cluster the coefficient has to be determined.
In Peacemaker the generation of the $N_c$ linear independend equations is done by iterating $N_c$ times over the calculation of the coefficients. In the first round (for the first equation) $Z_c$ is one for all components. In the second round $Z_c$ is one for all components but two for the second component. This is done until $Z_c$ is one for all components but $K$ for the last component. Then the calculated coefficents are sorted into an array and solved by the $\textbf{newton}$ subroutine. </p>

## Newton Algorithm to solve the Population Polynomial - Test

### Helper for finding the order of the polynomial coefficients in the array

In [17]:
# Array telling which coefficient belongs to which index
def coeff_order(degree):
    # degree           : input         - Array containing the highest degree of each monomer
    
    coeff = np.zeros((np.product(degree), len(degree)))
    
    if len(degree) == 1:
        for i in range(degree[0]):
            coeff[i] = i
            coeff = coeff.astype(int)
        return coeff
    elif len(degree) == 2:
        for i in range(degree[0]):
            for j in range(degree[1]):
                coeff[i*(degree[1])+j] = [i,j]
                coeff = coeff.astype(int)
    elif len(degree) == 3:
        for i in range(degree[0]):
            for j in range(degree[1]):
                for k in range(degree[2]):
                    coeff[i*degree[1]+j*degree[2]+j+k] = [i,j,k]
                    coeff = coeff.astype(int)
    elif len(degree) == 4:
        for i in range(degree[0]):
            for j in range(degree[1]):
                for k in range(degree[2]):
                    for l in range(degree[3]):
                        coeff[i+j*(degree[0])+k*(degree[0])*(degree[1])+l*(degree[0])*(degree[1])*(degree[2])] = [i,j,k,l]
                        coeff = coeff.astype(int)
    else: 
        print("The number of monomers is too high")
    

    # n_comp is the number of possible combinations of the monomers in the system
    n_comp = 1
    for i in range(1, len(degree)+1):
        n_comp = n_comp * degree[i-1]

    # coeff_order is the array containing the degree of each monomer for all the possible combinations
    # and all equations. It has the size of [n_comp*nr_monomers, nr_monomers]
    coeff_order = np.zeros((np.product(degree)*len(degree), len(degree)))

    # Loop over all clusters and set the coefficients
    for cluster in range(len(coeff)):
        index = coeff[cluster][0]

        for j in range(1, len(degree)):
            index += coeff[cluster][j] * np.product(degree[:j])

        for k in range(0, len(degree)):
            coeff_order[index + k*np.product(degree)] += coeff[cluster]
    
    return coeff_order

####  Degree = Highest population of the monomer in the clusterset + 1
##### One monomer - highest degree [4]

In [18]:
degree = []
degree = [5]
coeffs_all = coeff_order(degree)

print("Length of coefficient array: " , len(coeffs_all))
print("coefficient array: ", "\n", coeffs_all[0:5])

Length of coefficient array:  5
coefficient array:  
 [[0]
 [1]
 [2]
 [3]
 [4]]


##### Two monomers - highest degree [3,3]

In [19]:
degree = (4,4)
coeffs_all = coeff_order(degree)

print("Length of coefficient array: ",len(coeffs_all))
print("coefficient array: ", "\n", coeffs_all[16:32])

Length of coefficient array:  32
coefficient array:  
 [[0. 0.]
 [1. 0.]
 [2. 0.]
 [3. 0.]
 [0. 1.]
 [1. 1.]
 [2. 1.]
 [3. 1.]
 [0. 2.]
 [1. 2.]
 [2. 2.]
 [3. 2.]
 [0. 3.]
 [1. 3.]
 [2. 3.]
 [3. 3.]]


##### Three monomers - highest degree [3,2,3]

In [20]:
degree = (4,3,4)
coeffs_all = coeff_order(degree)

print("Length of coefficient array: ", len(coeffs_all))
array = np.array([i for i in range(1, 145)])
coeffs_all = np.column_stack((array, coeffs_all))
print("coefficient array: ", "\n", coeffs_all[0:48])

Length of coefficient array:  144
coefficient array:  
 [[ 1.  0.  0.  0.]
 [ 2.  1.  0.  0.]
 [ 3.  2.  0.  0.]
 [ 4.  3.  0.  0.]
 [ 5.  0.  0.  0.]
 [ 6.  0.  0.  0.]
 [ 7.  0.  0.  0.]
 [ 8.  3.  1.  0.]
 [ 9.  0.  0.  0.]
 [10.  0.  0.  0.]
 [11.  0.  0.  0.]
 [12.  3.  2.  0.]
 [13.  0.  0.  1.]
 [14.  1.  0.  1.]
 [15.  2.  0.  1.]
 [16.  3.  0.  1.]
 [17.  0.  0.  0.]
 [18.  0.  0.  0.]
 [19.  0.  0.  0.]
 [20.  3.  1.  1.]
 [21.  0.  0.  0.]
 [22.  0.  0.  0.]
 [23.  0.  0.  0.]
 [24.  3.  2.  1.]
 [25.  0.  0.  2.]
 [26.  1.  0.  2.]
 [27.  2.  0.  2.]
 [28.  3.  0.  2.]
 [29.  0.  0.  0.]
 [30.  0.  0.  0.]
 [31.  2.  1.  2.]
 [32.  3.  1.  2.]
 [33.  0.  0.  0.]
 [34.  0.  0.  0.]
 [35.  2.  2.  2.]
 [36.  3.  2.  2.]
 [37.  0.  0.  0.]
 [38.  0.  0.  0.]
 [39.  0.  0.  0.]
 [40.  3.  0.  3.]
 [41.  0.  0.  0.]
 [42.  0.  0.  0.]
 [43.  0.  0.  0.]
 [44.  3.  1.  3.]
 [45.  0.  0.  0.]
 [46.  0.  0.  0.]
 [47.  0.  0.  0.]
 [48.  3.  2.  3.]]


##### Four monomers - highest degree [4,1,2,2]

In [21]:
degree = (5,2,3,3)
coeffs_all = coeff_order(degree)

print("Length of coefficient array: ", len(coeffs_all))
array = np.array([i for i in range(1, 361)])
coeffs_all = np.column_stack((array, coeffs_all))
print("coefficient array: ", "\n", coeffs_all[0:90])

Length of coefficient array:  360
coefficient array:  
 [[ 1.  0.  0.  0.  0.]
 [ 2.  1.  0.  0.  0.]
 [ 3.  2.  0.  0.  0.]
 [ 4.  3.  0.  0.  0.]
 [ 5.  4.  0.  0.  0.]
 [ 6.  0.  1.  0.  0.]
 [ 7.  1.  1.  0.  0.]
 [ 8.  2.  1.  0.  0.]
 [ 9.  3.  1.  0.  0.]
 [10.  4.  1.  0.  0.]
 [11.  0.  0.  1.  0.]
 [12.  1.  0.  1.  0.]
 [13.  2.  0.  1.  0.]
 [14.  3.  0.  1.  0.]
 [15.  4.  0.  1.  0.]
 [16.  0.  1.  1.  0.]
 [17.  1.  1.  1.  0.]
 [18.  2.  1.  1.  0.]
 [19.  3.  1.  1.  0.]
 [20.  4.  1.  1.  0.]
 [21.  0.  0.  2.  0.]
 [22.  1.  0.  2.  0.]
 [23.  2.  0.  2.  0.]
 [24.  3.  0.  2.  0.]
 [25.  4.  0.  2.  0.]
 [26.  0.  1.  2.  0.]
 [27.  1.  1.  2.  0.]
 [28.  2.  1.  2.  0.]
 [29.  3.  1.  2.  0.]
 [30.  4.  1.  2.  0.]
 [31.  0.  0.  0.  1.]
 [32.  1.  0.  0.  1.]
 [33.  2.  0.  0.  1.]
 [34.  3.  0.  0.  1.]
 [35.  4.  0.  0.  1.]
 [36.  0.  1.  0.  1.]
 [37.  1.  1.  0.  1.]
 [38.  2.  1.  0.  1.]
 [39.  3.  1.  0.  1.]
 [40.  4.  1.  0.  1.]
 [41.  0.  0.  1.  1.]
 

### Solve polynomials with sympy

Example polynomials: </p>
$ f(x,y) = -1 + x + y $ </br>
$ g(x,y) = -3 + 2x + 4y $ </p> 

Solution: $[x, y] = [0.5, 0.5]$ </p>

In [22]:
x,y = sp.symbols('x, y')
eqs = [x + y -1, -3 + 2*x + 4*y]
sol = sp.solve(eqs, (x, y))
print(sol)

{x: 1/2, y: 1/2}


Example polynomials: </p>
$ f(x,y) = -1 + x^2 + y^2 $ </br>
$ g(x,y) = 2x - y $ </p> 

Solution: $[x, y] = [ \frac{1}{\sqrt{5}},  \frac{2}{\sqrt{5}}$] </p>
$~~~~~~~~~~~~~~~~~~~~~~~~~~~\Bigg( [- \frac{1}{\sqrt{5}}, - \frac{2}{\sqrt{5}}] \Bigg)$ </p>

In [23]:
x,y = sp.symbols('x, y')
eqs = [x**2 + y**2 -1, 2*x - y]
sol = sp.solve(eqs, (x, y))

#Show only the with x and y between 0 and 1
sol = [s for s in sol if 0 <= s[0] <= 1 and 0 <= s[1] <= 1]
print(sol)

[(sqrt(5)/5, 2*sqrt(5)/5)]


Example polynomials: </p>
$ f(x,y) = 2 + 3x - x^2 + 5x^3 - 15y + 2y^2 + 8y^3 + 20xy + x^2y + 12xy^2 + 100x^2y^2 + 1000x^3y^2 + 598x^2y^3 - 2105x^3y^3$ </br>
$ g(x,y) = x + x^2 + 2x^3 + y - 5y^2 -27.7xy^2 - 3x^2y^2 $ </p> 

Solution: $[x, y] = [0.1, 0.2]$ 

In [24]:
np.set_printoptions(precision=12)
x,y = sp.symbols('x, y')
eqs = [2 + 3*x - x**2 + 5*x**3 - 15*y + 2*y**2 + 8*y**3 + 20*x*y + x**2*y + 12*x*y**2 + 100*x**2*y**2 + 1000*x**3*y**2 + 598*x**2*y**3 - 2105*x**3*y**3, x + x**2 + 2*x**3 + y - 5*y**2 -27.7*x*y**2 - 3*x**2*y**2 ]
sol = sp.solve(eqs, (x, y))

#Show only the with x and y between 0 and 1
sol = [s for s in sol if 0 <= s[0] <= 1 and 0 <= s[1] <= 1]
print(sol)

[(0.100000000000000, 0.200000000000000)]


Example polynomials: </p>
$ f(x,y,z) = -1.2568 + x + x^2 + 5x^3 - y + 3y^2 + 4z + xy + xyz^2$ </br>
$ g(x,y,z) = -0.3129 + x^3 + 8y^2 - z^3 + yz^2 + 3x^3z $ </br> 
$ h(x,y,z) = 0.983 + 5x^3 - y + 8y^2 - 4z + z^2 + x^2y $ </p>

Solution: $[x, y, z] = [0.1, 0.2, 0.3]$ 

In [25]:
#x,y,z= sp.symbols('x, y, z')
#eqs = [-1.2568 + x + x**2 + 5*x**3 - y + 3*y**2 + 4*z + x*y + x*y*z**2, -0.3129 + x**3 + 8*y**2 - z**3 + y*z**2 + 3*x**3*z, 0.983 + 5*x**3 - y + 8*y**2 - 4*z + z**2 + x**2*y]
#sol = sp.solve(eqs, (x, y, z))
#
##Show only the with x and y between 0 and 1
#sol = [s for s in sol if 0 <= s[0] <= 1 and 0 <= s[1] <= 1]
#print(sol)

Example polynomials: </p>
$ f(a, b, c, d) =  0.61790213 + a + 2a^2 + 3a^3 -5a^4 + ab - 2abcd - 5bc^2 - 8cd $ </br>
$ g(a, b, c, d) = 0.3488084515 + a^3 - 4b + 6ac^2 + 5a^3bd^2 + 5bd $ </br>
$ h(a, b, c, d) = -0.17944167 + 3a^2 -a^3 + a^3b + bcd^2 -c^2d^2 $ </br>
$ i(a, b, c, d) = -0.1328316336 + a - a^2b + 8bc^2d^2 - 7c^2d + 6a^3d^2 $ </p>

Solution: $[a, ~b, ~c, ~d] = [0.23,~ 0.53,~ 0.19,~ 0.64]$ 

In [26]:
#a,b,c,d = sp.symbols('a,b,c,d')
#eqs = [0.61790213 + a + 2*a**2 + 3*a**3 -5*a**4 + a*b - 2*a*b*c*d - 5*b*c**2 - 8*c*d, 0.3488084515 + a**3 - 4*b + 6*a*c**2 + 5*a**3*b*d**2 + 5*b*d, -0.17944167 + 3*a**2 -a**3 + a**3*b + b*c*d**2 -c**2*d**2, 
#       -0.1328316336 + a - a**2*b + 8*b*c**2*d**2 - 7*c**2*d + 6*a**3*d**2]
#sol = sp.solve(eqs, (a,b,c,d))
#
##Show only the with x and y between 0 and 1
#sol = [s for s in sol if 0 <= s[0] <= 1 and 0 <= s[1] <= 1]
#print(sol)

### Horner

In [27]:
np.set_printoptions(precision=12)
x = 0.2000000000000
y = 0.3000000000000
f = -2 + x + x**2 + 7*x**3 + 2*y + x*y + 8*x**2*y - 2*x**3*y - 5*y**2 + x*y**2 - x**2*y**2 + 3*x**3*y**2 - y**3 + x*y**3 + x**2*y**3 + 12*x**3*y**3
print(f)

# detivative of f with respect to x
f_x = 1 + 2*x + 21*x**2 + y + 16*x*y - 6*x**2*y + y**2 - 2*x*y**2 + 9*x**2*y**2 + y**3 + 2*x*y**3 + 36*x**2*y**3
print(f_x)
# detivative of f with respect to y
f_y = 2 + x + 8*x**2 - 2*x**3 - 10*y + 2*x*y - 2*x**2*y + 6*x**3*y - 3*y**2 + 3*x*y**2 + 3*x**2*y**2 + 36*x**3*y**2
print(f_y)

-1.404168
3.59108
-0.5648799999999994


# Calculation of the remaining coefficients
$$ \begin{aligned} N_i &= q_i \prod_{c=1}^{K} \bigg( \frac{N_c}{q_c} \bigg)^{n_i^c} \\
   ln(N_i) &= ln(q_i) + \sum_{c=1}^K n_i^c \cdot  \bigg( \ln{N_c} - \ln{q_c} \bigg) \end{aligned}$$

In [75]:
def calculate_remaining_coefficents(monomer_populations, populations, lnq, clusterset, monomer):
    # monomer_populations : input         - Array containing the monomer populations (length = number of monomers)
    # populations         : input/output  - Array containing the populations of all the clusters (length = number of clusters)
    # lnq                 : input         - Array containing the natural logarithm of the partition function
    # lnq_monomer         : input         - Array containing the natural logarithm of the partition function of the monomers
    # clusterset          : input         - Array containing the clusters
    
    for i in range(len(clusterset)):
        populations[i] = lnq[i]
        for c in range(len(monomer_populations)):
            populations[i] += clusterset[i][c] * (np.log(monomer_populations[c]) - lnq[monomer[c]])
        populations[i] = np.exp(populations[i])
    
    return populations

#### Tests

In [76]:
clusterset = [[1, 0, 0], [1, 2, 0], [0, 1, 0], [1, 1, 1], [2, 1, 3], [0, 0, 1]]
populations = np.zeros(len(clusterset))
monomer_populations = [0.01, 0.2, 0.03]
lnq = [1.2, 3.2, 1.3, 1.2, 0.5, 1.1]
monomer = [0, 2, 5]

populations = calculate_remaining_coefficents(monomer_populations, populations, lnq, clusterset, monomer)
print(populations)

[1.000000000000e-02 2.195246544376e-04 2.000000000000e-01
 5.443077197365e-06 8.118571642079e-13 3.000000000000e-02]


In [103]:
clusterset = [[1, 0, 0, 0], [1, 2, 0, 5], [0, 1, 40, 2], [1, 1, 0, 1], [0, 1, 0, 0], [2, 0, 3, 1], [0, 0, 1, 0], [2, 3, 1, 1], [0, 0, 0, 1]]
populations = np.zeros(len(clusterset))
monomer_populations = [2.0e23, 1.2e23, 3.1e22, 5.4e20]
lnq = [-3.2, 5.0, 25, 36.2, 4.0, 28.3, 0.8, 18.9, 1.0]
monomer = [0, 4, 6, 8]

populations = calculate_remaining_coefficents(monomer_populations, populations, lnq, clusterset, monomer)
print(populations)

[2.000000000000e+023 1.088297856368e+171                 inf
 1.128067738370e+082 1.200000000000e+023 2.523225797410e+148
 3.100000000000e+022 1.142209387045e+164 5.400000000000e+020]


/tmp/ipykernel_9431/2948228026.py:12: RuntimeWarning: overflow encountered in exp
  populations[i] = np.exp(populations[i])
